<a href="https://colab.research.google.com/github/chanseongparkk/2022_Data_Analysis_Intern_practice/blob/main/07_11_01_%ED%91%9C%EB%B3%B8%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 필요 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np

### 2. 데이터셋 생성

In [ ]:
X_value = np.arange(40).reshape(20,2)
y_value = np.arange(20)

sample_df = pd.DataFrame(np.column_stack((X_value, y_value)),columns = ["X_1", "X_2", "result"])

display(sample_df)

sample_df.head()

,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2
3,6,7,3
4,8,9,4
5,10,11,5
6,12,13,6
7,14,15,7
8,16,17,8
9,18,19,9


,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2
3,6,7,3
4,8,9,4


### 3. 표본추출

#### 3-1 단순확률표본추출
- pandas 모듈의 DataFrame.sample() 메소드 이용

In [ ]:
sample_df.sample(n = 5, random_state = 1001) # sample_df에서 5개 랜덤추출

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18


In [ ]:
sample_df.sample(frac = 0.5, random_state = 1001) # sample_df에서 50%를 랜덤추출한다.

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2
18,36,37,18
7,14,15,7
10,20,21,10
6,12,13,6
19,38,39,19
4,8,9,4


In [ ]:
# 복원 무작위 표본추출

rep_df = sample_df.sample(frac = 0.2, random_state = 1001) # 샘플 4개 추출
display(rep_df.head())

# replace = True : 복원 추출

rep_df.sample(n = 10, replace = True, random_state = 1001)

,X_1,X_2,result
1,2,3,1
15,30,31,15
0,0,1,0
2,4,5,2


,X_1,X_2,result
15,30,31,15
15,30,31,15
15,30,31,15
0,0,1,0
0,0,1,0
2,4,5,2
1,2,3,1
1,2,3,1
0,0,1,0
2,4,5,2


In [ ]:
# 특정 칼럼의 값을 기준으로 가중치를 부여하여 무작위 표본 추출
# weights: 가중치 반영한 필드값
sample_df.sample(n = 5, weights = "result")

,X_1,X_2,result
13,26,27,13
4,8,9,4
17,34,35,17
7,14,15,7
15,30,31,15


#### 3-2 계통표본추출
- sysmetic_sample 함수 정의

In [ ]:
# 계통표본추출 함수 정의
def sysmetic_sampling(data, n):  #모집단 데이터프레임과 추출할 샘플 수
  count = len(data)
  sample_count = count // n      # 구간 내 필요한 샘플 수
  index = data[:sample_count].sample(1).index
  intoin = index - 0            #샘플 간 간격
  sys_df = pd.DataFrame()
  while len(sys_df) < n:
    sys_df = sys_df.append(data.loc[index,:])
    index += sample_count
  return sys_df

In [ ]:
# 구간 내 필요 샘플 수로 간격 정의
# 함수 호출

sysmetic_sampling(sample_df, 5)

,X_1,X_2,result
2,4,5,2
6,12,13,6
10,20,21,10
14,28,29,14
18,36,37,18


#### 3-3 층화확률표본추출
- sklearn의 StratifiedShuffleSplit(n_splits: 분할반복 횟수, test_size: 테스트셋 샘플 비율)s

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitfi = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = 1001)

In [ ]:
# 현재 데이터 확인
sample_df.head()

,X_1,X_2,result
0,0,1,0
1,2,3,1
2,4,5,2
3,6,7,3
4,8,9,4


In [ ]:
# 층을 구분하기 위한 값(0: 그룹 0, 1: 그룹 1)
group = [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]
sample_df["group"] = group

In [ ]:
# 데이터 전체 수 확인
sample_df.group.value_counts()

0    10
1    10
Name: group, dtype: int64

In [ ]:
# df_strat_train, df_strat_test로 데이터 분할(표본추출)
for train_idx, test_idx in splitfi.split(sample_df, sample_df.group):
  print("Train :", train_idx, "Test :", test_idx)
  df_strat_train = sample_df.loc[train_idx]
  df_strat_test = sample_df.loc[test_idx] 


Train : [13  7  1 14 16 12  0 11 10 18  2  8  5  6] Test : [17 19  3 15  4  9]


In [ ]:
print("Train data 수 :", df_strat_train.shape,"\n",
      "Test data 수 :", df_strat_test.shape)

Train data 수 : (14, 4) 
 Test data 수 : (6, 4)


In [ ]:
# 모집단과 동일 비율로 Group 속성을 기준으로 데이터 분리 확인
print("전제 비율","\n",
      sample_df.group.value_counts()/len(sample_df),"\n",
      "Train data 비율", "\n",
      df_strat_train.group.value_counts()/len(df_strat_train),"\n",
      "Test data 비율", "\n",
      df_strat_test.group.value_counts()/len(df_strat_test))

전제 비율 
 0    0.5
1    0.5
Name: group, dtype: float64 
 Train data 비율 
 1    0.5
0    0.5
Name: group, dtype: float64 
 Test data 비율 
 1    0.5
0    0.5
Name: group, dtype: float64
